# Tic-Tac-Toe Reinforcement Learning: Training and Evaluating Optimal Policies

# Import necessary libraries

In [ ]:
!pip install prettytable

In [1]:
import numpy as np
import pickle
from collections import defaultdict

# Load perfect policy

In [2]:
perfectPolicy = pickle.load(open("perfectPolicy.p", "rb"))

In [3]:
len(perfectPolicy)

4520

In [27]:
perfectPolicy

{(0, 0, 0, 0, 0, 0, 0, 0, 0): array([0., 1., 0., 0., 0., 0., 0., 0., 0.]),
 (1, 0, 0, 0, 0, 0, 0, 0, 0): array([0., 0., 0., 1., 0., 0., 0., 0.]),
 (1, 2, 0, 0, 0, 0, 0, 0, 0): array([0., 0., 1., 0., 0., 0., 0.]),
 (1, 2, 1, 0, 0, 0, 0, 0, 0): array([0., 1., 0., 0., 0., 0.]),
 (1, 2, 1, 2, 0, 0, 0, 0, 0): array([1., 0., 0., 0., 0.]),
 (1, 2, 1, 2, 1, 0, 0, 0, 0): array([1., 0., 0., 0.]),
 (1, 2, 1, 2, 1, 2, 0, 0, 0): array([1., 0., 0.]),
 (1, 2, 1, 2, 1, 2, 0, 1, 0): array([1., 0.]),
 (1, 2, 1, 2, 1, 2, 2, 1, 0): array([1.]),
 (1, 2, 1, 2, 1, 2, 0, 1, 2): array([1.]),
 (1, 2, 1, 2, 1, 0, 2, 0, 0): array([0., 0., 1.]),
 (1, 2, 1, 2, 1, 1, 2, 0, 0): array([0., 1.]),
 (1, 2, 1, 2, 1, 1, 2, 2, 0): array([1.]),
 (1, 2, 1, 2, 1, 1, 2, 0, 2): array([1.]),
 (1, 2, 1, 2, 1, 0, 2, 1, 0): array([0., 1.]),
 (1, 2, 1, 2, 1, 0, 2, 1, 2): array([1.]),
 (1, 2, 1, 2, 1, 0, 0, 2, 0): array([1., 0., 0.]),
 (1, 2, 1, 2, 1, 1, 0, 2, 0): array([1., 0.]),
 (1, 2, 1, 2, 1, 1, 0, 2, 2): array([1.]),
 (1, 2, 1, 

# Create random policy

In [4]:
randomPolicy = dict()
for board in perfectPolicy:
    l = len(perfectPolicy[board])
    randomPolicy[board] = np.ones(l) / l
    print(board,randomPolicy[board])

(0, 0, 0, 0, 0, 0, 0, 0, 0) [0.11111111 0.11111111 0.11111111 0.11111111 0.11111111 0.11111111
 0.11111111 0.11111111 0.11111111]
(1, 0, 0, 0, 0, 0, 0, 0, 0) [0.125 0.125 0.125 0.125 0.125 0.125 0.125 0.125]
(1, 2, 0, 0, 0, 0, 0, 0, 0) [0.14285714 0.14285714 0.14285714 0.14285714 0.14285714 0.14285714
 0.14285714]
(1, 2, 1, 0, 0, 0, 0, 0, 0) [0.16666667 0.16666667 0.16666667 0.16666667 0.16666667 0.16666667]
(1, 2, 1, 2, 0, 0, 0, 0, 0) [0.2 0.2 0.2 0.2 0.2]
(1, 2, 1, 2, 1, 0, 0, 0, 0) [0.25 0.25 0.25 0.25]
(1, 2, 1, 2, 1, 2, 0, 0, 0) [0.33333333 0.33333333 0.33333333]
(1, 2, 1, 2, 1, 2, 0, 1, 0) [0.5 0.5]
(1, 2, 1, 2, 1, 2, 2, 1, 0) [1.]
(1, 2, 1, 2, 1, 2, 0, 1, 2) [1.]
(1, 2, 1, 2, 1, 0, 2, 0, 0) [0.33333333 0.33333333 0.33333333]
(1, 2, 1, 2, 1, 1, 2, 0, 0) [0.5 0.5]
(1, 2, 1, 2, 1, 1, 2, 2, 0) [1.]
(1, 2, 1, 2, 1, 1, 2, 0, 2) [1.]
(1, 2, 1, 2, 1, 0, 2, 1, 0) [0.5 0.5]
(1, 2, 1, 2, 1, 0, 2, 1, 2) [1.]
(1, 2, 1, 2, 1, 0, 0, 2, 0) [0.33333333 0.33333333 0.33333333]
(1, 2, 1, 2, 1, 1, 0

# Basic Framework

In [5]:
def create_board():
    """Create an empty 3x3 Tic-Tac-Toe board."""
    return [0] * 9  # 0 represents an empty cell

def display_board(board):
    """Display the Tic-Tac-Toe board."""
    symbols = [' ', 'O', 'X']
    for i in range(0, 9, 3):
        print(f"{symbols[board[i]]} | {symbols[board[i+1]]} | {symbols[board[i+2]]}")
        if i < 6:
            print("--+---+--")
    print()

def is_valid_move(board, move):
    """Check if a move is valid."""
    return board[move] == 0

def make_move(board, move, player):
    """Make a move on the board."""
    new_board = board[:]
    new_board[move] = player
    return new_board

def check_winner(board):
    """Check if there's a winner or if the game is a draw."""
    win_positions = [
        (0, 1, 2), (3, 4, 5), (6, 7, 8),  # Rows
        (0, 3, 6), (1, 4, 7), (2, 5, 8),  # Columns
        (0, 4, 8), (2, 4, 6)              # Diagonals
    ]
    for p1, p2, p3 in win_positions:
        if board[p1] == board[p2] == board[p3] and board[p1] != 0:
            return board[p1]  # Return the winning player (1 or 2)
    if 0 not in board:
        return 0  # Draw
    return -1  # Game is ongoing


# Test the basic framework

# if __name__ == "__main__":
    board = create_board()
    current_player = 1
    while check_winner(board) == -1:
        display_board(board)
        move = int(input(f"Player {current_player} (O for 1, X for 2), choose your move (0-8): "))
        if is_valid_move(board, move):
            board = make_move(board, move, current_player)
            current_player = 3 - current_player  # Switch player
        else:
            print("Invalid move. Try again.")
    display_board(board)
    winner = check_winner(board)
    if winner == 0:
        print("It's a draw!")
    else:
        print(f"Player {winner} wins!")


# Define functions for Tic-Tac-Toe mechanics

In [6]:
def stateOfBoard(board):
    """Check the current state of the board."""
    winning_lines = [
        (0, 1, 2), (3, 4, 5), (6, 7, 8),  # rows
        (0, 3, 6), (1, 4, 7), (2, 5, 8),  # columns
        (0, 4, 8), (2, 4, 6)              # diagonals
    ]
    for a, b, c in winning_lines:
        if board[a] == board[b] == board[c] != 0:
            return board[a]  # 1 if O won, 2 if X won
    if 0 in board:
        return -1  # The game is not finished yet
    return 0  # Draw

def makeMove(board, location, player):
    """Apply a move to the board."""
    board = list(board)
    board[location] = player
    return tuple(board)

def getEmpty(board):
    """Return the list of empty locations on the board."""
    return [i for i in range(9) if board[i] == 0]

def showBoard(board):
    """Display the board in a readable format."""
    S = [".", "O", "X"]
    print(S[board[0]], S[board[1]], S[board[2]])
    print(S[board[3]], S[board[4]], S[board[5]])
    print(S[board[6]], S[board[7]], S[board[8]])

def play(policyA, policyB, visualize=False):
    """Simulate a game between two policies."""
    board = (0, 0, 0, 0, 0, 0, 0, 0, 0)
    nextPlayer = [0, 2, 1]
    player = 1
    history = []
    while stateOfBoard(board) == -1:
        locations = getEmpty(board)

        if player == 1:
            # Filter probabilities to match available locations
            valid_policy = np.zeros(9)  # Initialize with zeros for all positions
            for i, loc in enumerate(locations):
                valid_policy[loc] = policyA[board][i]
            valid_policy /= np.sum(valid_policy)  # Normalize to sum to 1
            chosenLocation = np.random.choice(locations, p=valid_policy[locations])
        else:
            # Same for the opponent's policy
            valid_policy = np.zeros(9)
            for i, loc in enumerate(locations):
                valid_policy[loc] = policyB[board][i]
            valid_policy /= np.sum(valid_policy)
            chosenLocation = np.random.choice(locations, p=valid_policy[locations])
        history.append([board, locations.index(chosenLocation)])
        board = makeMove(board, chosenLocation, player)
        player = nextPlayer[player]
        if visualize:
            showBoard(board)
            print("----")
    return stateOfBoard(board), history



# Train a random policy against the perfect policy

In [7]:
collector = [0, 0, 0]  # Track wins, losses, draws
for i in range(10000):
    outcome, _ = play(randomPolicy, perfectPolicy)
    collector[outcome] += 1

print("Results against perfect policy:", collector)

Results against perfect policy: [2327, 0, 7673]


# Training

In [8]:
def train_rl(policy, episodes=10000, alpha=0.1, gamma=0.9):
    Q = {}  # Initialize Q-values for state-action pairs

    for episode in range(episodes):
        board = (0, 0, 0, 0, 0, 0, 0, 0, 0)  # Reset board for each episode
        player = 1
        history = []

        while stateOfBoard(board) == -1:
            locations = getEmpty(board)

            # Initialize Q-values and policy for new boards
            if board not in Q:
                Q[board] = np.zeros(9)  # Initialize Q-values for all positions
            if board not in policy:
                policy[board] = np.ones(len(locations)) / len(locations)

            # Filter policy to match available locations
            valid_policy = np.zeros(9)  # Start with zero probabilities for all positions
            for i, loc in enumerate(locations):
                valid_policy[loc] = policy[board][i]

            # Normalize the filtered policy to ensure probabilities sum to 1
            valid_policy /= np.sum(valid_policy)

            # Choose action based on the valid policy
            action = np.random.choice(locations, p=valid_policy[locations])
            history.append((board, action))
            board = makeMove(board, action, player)

            # Switch player
            player = 3 - player  # Toggle between 1 and 2

        # Backpropagate rewards
        outcome = stateOfBoard(board)
        reward = 1 if outcome == 1 else -1 if outcome == 2 else 0

        for state, action in reversed(history):
            if state not in Q:
                Q[state] = np.zeros(9)

            Q[state][action] += alpha * (reward + gamma * max(Q[state]) - Q[state][action])
            policy[state] = np.exp(Q[state]) / np.sum(np.exp(Q[state]))

    return policy


# Evaluate the trained policy

In [9]:
trainedPolicy = train_rl(randomPolicy, 10000)

In [10]:
trainedPolicy

{(0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0): array([0.08220679, 0.11951419, 0.10583463, 0.11927776, 0.13802764,
        0.08368604, 0.13099844, 0.09800685, 0.12244767]),
 (1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0): array([0.00206919, 0.02556945, 0.07907548, 0.13924053, 0.14306472,
        0.13471628, 0.14070168, 0.18426402, 0.15129864]),
 (1,
  2,
  0,
  0,
  0,
  0,
  0,
  0,
  0): array([0.09234966, 0.09234966, 0.09220649, 0.11968651, 0.1057885 ,
        0.13127711, 0.09757388, 0.11735082, 0.15141737]),
 (1,
  2,
  1,
  0,
  0,
  0,
  0,
  0,
  0): array([0.09992019, 0.09992019, 0.09992019, 0.12177605, 0.10822123,
        0.15223341, 0.12708886, 0.09774504, 0.09317483]),
 (1,
  2,
  1,
  2,
  0,
  0,
  0,
  0,
  0): array([0.10650879, 0.10650879, 0.10650879, 0.10650879, 0.10650879,
        0.10650879, 0.1299601 , 0.12057313, 0.11041406]),
 (1,
  2,
  1,
  2,
  1,
  0,
  0,
  0,
  0): array([0.10431945, 0.10431945, 0.10431945, 0.10431945, 0.10431945,
        0.10431945, 0.11620639, 

In [11]:
print("trainedPolicy board: ")
trainedPolicy[board]

trainedPolicy board: 


array([0.1683895 , 0.18314889, 0.1640601 , 0.12303425, 0.15092784,
       0.1028165 , 0.06822939, 0.01969677, 0.01969677])

In [12]:
print("perfectPolicy board: ")
perfectPolicy[board]

perfectPolicy board: 


array([0., 0., 0., 0., 0., 1., 0.])

# Evaluate the trained policy

In [ ]:
collector = [0, 0, 0]
for i in range(10000):
    outcome, _ = play(trainedPolicy, perfectPolicy)
    collector[outcome] += 1

print("Trained policy results against perfect policy:", collector)